In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from imblearn.over_sampling import SMOTE

In [2]:
loan_result=pd.read_csv('loan_result.csv')
log_data=pd.read_csv('log_data.csv')
user_spec=pd.read_csv('user_spec.csv')

In [3]:
### 1번 ###
user_spec1=user_spec.copy()
loan_result1=loan_result.copy()

# 안 쓰는 변수 제거
user_spec1.drop(user_spec.columns[[4, 13, 14]], axis=1, inplace=True)
loan_result1.drop(loan_result.columns[[1, 2, 3]], axis=1, inplace=True)

# 근로연월 -> 연차
company_enter_month=user_spec1['company_enter_month'].astype(str)
working_year=2022-pd.DataFrame(map(int,company_enter_month.str[:4][company_enter_month.str[:4]!='nan']))

# 생년 -> 나이
birth_year=user_spec1['birth_year'].astype(str)
age=2023-pd.DataFrame(map(int,birth_year.str[:4][birth_year.str[:4]!='nan']))
working_year.index=company_enter_month.str[:4][company_enter_month.str[:4]!='nan'].index
age.index=birth_year.str[:4][birth_year.str[:4]!='nan'].index
user_spec1['company_enter_month']=working_year
user_spec1['birth_year']=age
user_spec1.rename(columns={'birth_year':'age', 'company_enter_month':'working_year'}, inplace=True)

# 미성년자 때부터 경력으로 입력한 자료 제거
user_spec1.drop(user_spec1[(user_spec1['age']-user_spec1['working_year'])<=18].index, inplace=True)

# purpose 변수 통일
user_spec1['purpose'] = user_spec1['purpose'].replace('사업자금', 'BUSINESS')
user_spec1['purpose'] = user_spec1['purpose'].replace('자동차구입', 'BUYCAR')
user_spec1['purpose'] = user_spec1['purpose'].replace('주택구입', 'BUYHOUSE')
user_spec1['purpose'] = user_spec1['purpose'].replace('기타', 'ETC')
user_spec1['purpose'] = user_spec1['purpose'].replace('전월세보증금', 'HOUSEDEPOSIT')
user_spec1['purpose'] = user_spec1['purpose'].replace('투자', 'INVEST')
user_spec1['purpose'] = user_spec1['purpose'].replace('생활비', 'LIVING')
user_spec1['purpose'] = user_spec1['purpose'].replace('대환대출', 'SWITCHLOAN')

# 공통 결측치가 있는 데이터와 결측치가 있는 데이터 중 수가 매우 적은 데이터 제거
user_spec1=user_spec1[user_spec1['age'].isnull()==False] #
user_spec1=user_spec1[user_spec1['yearly_income'].isnull()==False]

# 기대출수, 기대출금액이 모두 결측치면 0으로 대체
user_spec1.loc[user_spec1['existing_loan_cnt'].isnull()&user_spec1['existing_loan_amt'].isnull(),['existing_loan_cnt','existing_loan_amt']]=0

# 연소득이 0인데 연차가 결측치이면 연차를 0으로 대체
user_spec1.loc[(user_spec1['yearly_income']==0) & (user_spec1['working_year'].isnull()), 'working_year']=0

# 연소득이 0인데 연차가 존재하는 데이터 제거
user_spec1.drop(index=user_spec1[(user_spec1['yearly_income']==0) & (user_spec1['working_year']>=1)].index, inplace=True)

# 기타소득이 아니고 연차가 결측치인 데이터 제거
user_spec1.drop(index=user_spec1[(user_spec1['working_year'].isnull()) & (user_spec1['income_type']!='OTHERINCOME')].index, inplace=True)

# 기대출수 이상치 제거 (Q3+3*IQR 이상인 값을 이상치로 판단)
user_spec1=user_spec1[user_spec1['existing_loan_cnt']<28.5]

# 기대출금액이 0인데 기대출수가 0이 아닌 데이터 0으로 변환
user_spec1.loc[user_spec1['existing_loan_amt']==0,'existing_loan_cnt']=0

# 기대출금액 이상치 제거 (평균+5*표준편차를 초과하는 값을 이상치로 판단)
for i in user_spec1['existing_loan_cnt'].unique():
    a=user_spec1[user_spec1['existing_loan_cnt']==i]
    outlier_ind=a[a['existing_loan_amt']>(a['existing_loan_amt'].mean()+5*a['existing_loan_amt'].std())].index
    user_spec1.drop(index=outlier_ind, inplace=True)
    
# 대출희망금액 이상치 제거    
IQR=np.quantile(user_spec1['desired_amount'], 0.75) - np.quantile(user_spec1['desired_amount'], 0.25)
user_spec1.drop(index=user_spec1['desired_amount'][user_spec1['desired_amount']>np.quantile(user_spec1['desired_amount'], 0.75) + 3*IQR].index, inplace=True)

# 연소득 이상치 제거
IQR = np.quantile(user_spec1['yearly_income'], 0.75) - np.quantile(user_spec1['yearly_income'], 0.25)
user_spec1.drop(index=user_spec1['yearly_income'][user_spec1['yearly_income']>np.quantile(user_spec1['yearly_income'], 0.75) + 3*IQR].index, inplace=True)

# 신용점수랑 기대출 금액 결측값 제거한 데이터프레임
user_spec1_del=user_spec1.copy()
user_spec1_del.drop(index=user_spec1_del.loc[user_spec1_del['credit_score'].isnull() | user_spec1_del['existing_loan_amt'].isnull()].index, inplace=True)
    
# 기대출수가 1인 기대출 금액의 히스토그램을 보았을 때, 분포가 오른쪽으로 꼬리가 긴 그래프를 나타내고 있고, 
# 이상치가 존재하므로 중앙값으로 결측값을 대체(현재 기대출금액이 결측치인 데이터는 기대출수가 모두 1임)
user_spec1.loc[user_spec1['existing_loan_amt'].isnull() & (user_spec1['existing_loan_cnt']==1),'existing_loan_amt'] = user_spec1.loc[user_spec1['existing_loan_amt'].notnull() & (user_spec1['existing_loan_cnt']==1),'existing_loan_amt'].median()

# 신용점수 평균으로 결측값 대체
user_spec1.loc[user_spec1['credit_score'].isnull(),'credit_score'] = user_spec1['credit_score'][user_spec1['credit_score'].notnull()].mean()

# loan_limit, loan_late 결측값 제거
loan_result1.drop(index=loan_result1[loan_result1['loan_limit'].isnull()].index, inplace=True)

# loan_limit 이상치 제거
IQR = np.quantile(loan_result1['loan_limit'], 0.75) - np.quantile(loan_result1['loan_limit'], 0.25)
loan_result1.drop(index=loan_result1['loan_limit'][loan_result1['loan_limit']>np.quantile(loan_result1['loan_limit'], 0.75) + 3*IQR].index, inplace=True)

# loan_result1 copy
loan_result1_del=loan_result1.copy()

# user_spec1에서 loan_result1과 안 겹치는 application_id 제거
no_loan_app=list(set(user_spec1['application_id'])-set(loan_result1['application_id']))
user_spec1.set_index('application_id', inplace=True)
user_spec1.drop(index=no_loan_app, inplace=True)
user_spec1.reset_index(drop=False, inplace=True)

#loan_result1에서 user_spec1과 안 겹치는 application_id 제거
no_user_app=list(set(loan_result1['application_id'])-set(user_spec1['application_id']))
loan_result1.set_index('application_id', inplace=True)
loan_result1.drop(index=no_user_app, inplace=True)
loan_result1.reset_index(drop=False, inplace=True)

# user_spec1_del에서 loan_result1_del과 안 겹치는 application_id 제거
no_loan_app2=list(set(user_spec1_del['application_id'])-set(loan_result1_del['application_id']))
user_spec1_del.set_index('application_id', inplace=True)
user_spec1_del.drop(index=no_loan_app2, inplace=True)
user_spec1_del.reset_index(drop=False, inplace=True)

# loan_result1_del에서 user_spec1_del과 안 겹치는 application_id 제거
no_user_app2=list(set(loan_result1_del['application_id'])-set(user_spec1_del['application_id']))
loan_result1_del.set_index('application_id', inplace=True)
loan_result1_del.drop(index=no_user_app2, inplace=True)
loan_result1_del.reset_index(drop=False, inplace=True)

# 모델용 데이터
loan_result_model=loan_result1[loan_result1['is_applied'].notnull()]
loan_result_model_del=loan_result1_del[loan_result1_del['is_applied'].notnull()]

# user_spec1과 loan_result1 결합 / user_spec1_del과 loan_result1_del 결합
data1=user_spec1.merge(loan_result_model, on='application_id', how='inner')
data1_del=user_spec1_del.merge(loan_result_model_del, on='application_id', how='inner')

# 근로형태가 OTHERINCOME인 데이터는 연차를 빼고 분석하기 위해 데이터프레임을 나눔
data1_2=data1[data1['income_type']=='OTHERINCOME']
data1=data1[data1['income_type']!='OTHERINCOME']
data1_del_2=data1_del[data1_del['income_type']=='OTHERINCOME']
data1_del=data1_del[data1_del['income_type']!='OTHERINCOME']

# data1_2, data1_del_2에서 income_type, wokring_year 제거
data1_2.drop(columns=['income_type', 'working_year'], inplace=True)
data1_del_2.drop(columns=['income_type', 'working_year'], inplace=True)

# copy
data11=data1.copy()

# 더미변수 생성
data1=pd.get_dummies(data1, columns=['income_type', 'employment_type', 'houseown_type', 'purpose'])
data1_del=pd.get_dummies(data1_del, columns=['income_type', 'employment_type', 'houseown_type', 'purpose'])

### 2번 ###
log_data2=log_data.copy()
log_data2=log_data2[['user_id', 'event']]
log_data2=log_data2[(log_data2['event']!='SignUp') & (log_data2['event']!='OpenApp') & (log_data2['event']!='Login')]

In [8]:
# 결측치 제거, income_tpye=OTHERINCOME인 데이터 modeling
feature_columns = list(data1_del_2.columns.difference(['application_id', 'user_id', 'is_applied']))
x=data1_del_2[feature_columns]
y=data1_del_2['is_applied']
columns_n=['credit_score', 'yearly_income', 'desired_amount', 'existing_loan_cnt', 'existing_loan_amt', 'loan_rate', 'loan_limit']
xx=x[columns_n]
train_x, test_x, train_y, test_y = train_test_split(xx, y, train_size=0.8, random_state=2)

from imblearn.over_sampling import RandomOverSampler
xx_rs, y_rs=RandomOverSampler(random_state=0, sampling_strategy=3/7).fit_resample(train_x, train_y)

from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression(max_iter=100, penalty='l1', solver="liblinear")
logistic.fit(xx_rs, y_rs)

# confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

pred=logistic.predict(test_x)

test_cm = confusion_matrix(test_y, pred)
test_acc = accuracy_score(test_y, pred)
test_prc = precision_score(test_y, pred)
test_rcll = recall_score(test_y, pred)
test_f1 = f1_score(test_y, pred)

print(test_cm)
print('\n')
print('정확도\t{}%'.format(round(test_acc*100,2)))
print('정밀도\t{}%'.format(round(test_prc*100,2)))
print('재현율\t{}%'.format(round(test_rcll*100,2)))
print('F1\t{}%'.format(round(test_f1*100,2)))

[[45458  3559]
 [ 6889  2098]]


정확도	81.99%
정밀도	37.09%
재현율	23.34%
F1	28.65%


In [17]:
# 결측치 제거, income_type=FREELANCER modeling
xx = data1_del[data1_del['income_type_FREELANCER']==1][['credit_score','desired_amount', 'existing_loan_amt', 'existing_loan_cnt', 'loan_limit', 'loan_rate', 'working_year','yearly_income']].copy()
y = data1_del['is_applied'][data1_del['income_type_FREELANCER']==1]
train_x, test_x, train_y, test_y = train_test_split(xx, y, train_size=0.8, random_state=2)

from imblearn.over_sampling import RandomOverSampler
xx_rs, y_rs=RandomOverSampler(random_state=0, sampling_strategy=3/7).fit_resample(train_x, train_y)

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier

logistic=LogisticRegression(max_iter=100, penalty='l1', solver="liblinear")
logistic.fit(xx_rs, y_rs)

# confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

pred=logistic.predict(test_x)

test_cm = confusion_matrix(test_y, pred)
test_acc = accuracy_score(test_y, pred)
test_prc = precision_score(test_y, pred)
test_rcll = recall_score(test_y, pred)
test_f1 = f1_score(test_y, pred)

print(test_cm)
print('\n')
print('정확도\t{}%'.format(round(test_acc*100,2)))
print('정밀도\t{}%'.format(round(test_prc*100,2)))
print('재현율\t{}%'.format(round(test_rcll*100,2)))
print('F1\t{}%'.format(round(test_f1*100,2)))

[[28155  3100]
 [ 3130  1740]]


정확도	82.75%
정밀도	35.95%
재현율	35.73%
F1	35.84%


In [20]:
# 결측치 제거, income_type=PRIVATEBUSINESS modeling
xx = data1_del[data1_del['income_type_PRIVATEBUSINESS']==1][['credit_score','desired_amount', 'existing_loan_amt', 'existing_loan_cnt', 'loan_limit', 'loan_rate', 'working_year','yearly_income']].copy()
y = data1_del['is_applied'][data1_del['income_type_PRIVATEBUSINESS']==1]
train_x, test_x, train_y, test_y = train_test_split(xx, y, train_size=0.8, random_state=2)

from imblearn.over_sampling import RandomOverSampler
xx_rs, y_rs=RandomOverSampler(random_state=0, sampling_strategy=3/7).fit_resample(train_x, train_y)

from sklearn.linear_model import LogisticRegression

logistic=LogisticRegression(max_iter=100, penalty='l1', solver="liblinear")
logistic.fit(xx_rs, y_rs)

# confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

pred=logistic.predict(test_x)

test_cm = confusion_matrix(test_y, pred)
test_acc = accuracy_score(test_y, pred)
test_prc = precision_score(test_y, pred)
test_rcll = recall_score(test_y, pred)
test_f1 = f1_score(test_y, pred)

print(test_cm)
print('\n')
print('정확도\t{}%'.format(round(test_acc*100,2)))
print('정밀도\t{}%'.format(round(test_prc*100,2)))
print('재현율\t{}%'.format(round(test_rcll*100,2)))
print('F1\t{}%'.format(round(test_f1*100,2)))

[[38361  3994]
 [ 3154  2021]]


정확도	84.96%
정밀도	33.6%
재현율	39.05%
F1	36.12%


In [21]:
# 결측치 제거, income_type=PRACTITIONER modeling
xx = data1_del[data1_del['income_type_PRACTITIONER']==1][['credit_score','desired_amount', 'existing_loan_amt', 'existing_loan_cnt', 'loan_limit', 'loan_rate', 'working_year','yearly_income']].copy()
y = data1_del['is_applied'][data1_del['income_type_PRACTITIONER']==1]
train_x, test_x, train_y, test_y = train_test_split(xx, y, train_size=0.8, random_state=2)

from imblearn.over_sampling import RandomOverSampler
xx_rs, y_rs=RandomOverSampler(random_state=0, sampling_strategy=3/7).fit_resample(train_x, train_y)

from sklearn.linear_model import LogisticRegression

logistic=LogisticRegression(max_iter=100, penalty='l1', solver="liblinear")
logistic.fit(xx_rs, y_rs)

# confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

pred=logistic.predict(test_x)

test_cm = confusion_matrix(test_y, pred)
test_acc = accuracy_score(test_y, pred)
test_prc = precision_score(test_y, pred)
test_rcll = recall_score(test_y, pred)
test_f1 = f1_score(test_y, pred)

print(test_cm)
print('\n')
print('정확도\t{}%'.format(round(test_acc*100,2)))
print('정밀도\t{}%'.format(round(test_prc*100,2)))
print('재현율\t{}%'.format(round(test_rcll*100,2)))
print('F1\t{}%'.format(round(test_f1*100,2)))

[[16313  2073]
 [  772   528]]


정확도	85.55%
정밀도	20.3%
재현율	40.62%
F1	27.07%


In [29]:
# 결측치 제거, income_type=EARNEDINCOME or EARNEDINCOME2
xx = data1_del[(data1_del['income_type_EARNEDINCOME']==1) | (data1_del['income_type_EARNEDINCOME2']==1)][['credit_score','desired_amount', 'existing_loan_amt', 'existing_loan_cnt', 'loan_limit', 'loan_rate', 'working_year','yearly_income']].copy()
y = data1_del[(data1_del['income_type_EARNEDINCOME']==1) | (data1_del['income_type_EARNEDINCOME2']==1)]['is_applied']
train_x, test_x, train_y, test_y = train_test_split(xx, y, train_size=0.8, random_state=2)

from imblearn.over_sampling import RandomOverSampler
xx_rs, y_rs=RandomOverSampler(random_state=0, sampling_strategy=3/7).fit_resample(train_x, train_y)

from sklearn.linear_model import LogisticRegression

logistic=LogisticRegression(max_iter=100, penalty='l1', solver="liblinear")
logistic.fit(xx_rs, y_rs)

# confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

pred=logistic.predict(test_x)

test_cm = confusion_matrix(test_y, pred)
test_acc = accuracy_score(test_y, pred)
test_prc = precision_score(test_y, pred)
test_rcll = recall_score(test_y, pred)
test_f1 = f1_score(test_y, pred)

print(test_cm)
print('\n')
print('정확도\t{}%'.format(round(test_acc*100,2)))
print('정밀도\t{}%'.format(round(test_prc*100,2)))
print('재현율\t{}%'.format(round(test_rcll*100,2)))
print('F1\t{}%'.format(round(test_f1*100,2)))

[[1150720  133311]
 [  43088   26771]]


정확도	86.97%
정밀도	16.72%
재현율	38.32%
F1	23.29%


# 2번

In [ ]:
log_data2=log_data.copy()
log_data2=log_data2[['user_id', 'event']]
log_data2=log_data2[(log_data2['event']!='SignUp') & (log_data2['event']!='OpenApp') & (log_data2['event']!='Login')]

a=pd.get_dummies(log_data2, columns=['event'])
idx=[]
col=['CompleteIDCertification', 'EndLoanApply', 'GetCreditInfo', 'StartLoanApply', 
     'UseDSRCalc',  'UseLoanManage',  'UsePrepayCalc', 'ViewLoanApplyIntro']

for i in a.columns[1::]:
    idx.append(list(a[a[i]==1]['user_id'].value_counts().index))

aa=a.set_index('user_id')
for i in range(len(idx)):
    aa.loc[idx[i], col[i]]=a[a[aa.columns[i]]==1]['user_id'].value_counts()

aa=aa.loc[~aa.index.duplicated(keep='first')]
aa=aa.iloc[:, 8::]
aa=aa.fillna(0)